In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Copyright 2024 Google LLC.
# SPDX-License-Identifier: Apache-2.0

In [ ]:
!pip uninstall torch torchvision torchdata torchtext torch_xla libtpu-nightly -y

!pip3 install --pre torch torchvision torchtext --index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install https://storage.googleapis.com/pytorch-xla-releases/wheels/tpuvm/torch_xla-nightly-cp310-cp310-linux_x86_64.whl
!pip install torch_xla[tpu] -f https://storage.googleapis.com/libtpu-releases/index.html

Found existing installation: torch 2.5.0.dev20240629+cpu
Uninstalling torch-2.5.0.dev20240629+cpu:
  Successfully uninstalled torch-2.5.0.dev20240629+cpu
Found existing installation: torchvision 0.20.0.dev20240629+cpu
Uninstalling torchvision-0.20.0.dev20240629+cpu:
  Successfully uninstalled torchvision-0.20.0.dev20240629+cpu
Found existing installation: torchdata 0.7.1a0+b0e25e2
Uninstalling torchdata-0.7.1a0+b0e25e2:
  Successfully uninstalled torchdata-0.7.1a0+b0e25e2
Found existing installation: torchtext 0.17.0.dev20240629+cpu
Uninstalling torchtext-0.17.0.dev20240629+cpu:
  Successfully uninstalled torchtext-0.17.0.dev20240629+cpu
Found existing installation: torch-xla 2.4.0+git3bdca4d
Uninstalling torch-xla-2.4.0+git3bdca4d:
  Successfully uninstalled torch-xla-2.4.0+git3bdca4d
Found existing installation: libtpu-nightly 0.1.dev20240618+default
Uninstalling libtpu-nightly-0.1.dev20240618+default:
  Successfully uninstalled libtpu-nightly-0.1.dev20240618+default
Looking in index

In [ ]:
!git clone https://github.com/pytorch/data.git torchdata
%cd torchdata/
!git reset --hard b0e25e2
!pip install .
!cd ..

fatal: destination path 'torchdata' already exists and is not an empty directory.
/content/torchdata
HEAD is now at b0e25e26 Save state in dataset_iter_state when dataset is also an iterator (#1279)
Processing /content/torchdata
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torchdata: filename=torchdata-0.7.1a0+b0e25e2-py3-none-any.whl size=209379 sha256=6efdf9546495e5571448faf2f413c5455cfa654edb0ac7d0ed114aff07fccf08
  Stored in directory: /tmp/pip-ephem-wheel-cache-gzox3g1f/wheels/ec/25/35/f221f33eac65247616ab85a393ae829e700717149ec6f4d1ef
Successfully built torchdata


In [ ]:
!pip3 list | grep 'torch'

torch                        2.5.0.dev20240629+cpu
torch-xla                    2.4.0+git3bdca4d
torchaudio                   2.3.0+cpu
torchdata                    0.7.1a0+b0e25e2
torchtext                    0.17.0.dev20240629+cpu
torchvision                  0.20.0.dev20240629+cpu


In [ ]:
!pip install panda
!pip install pandas
!pip install 'portalocker>=2.0.0'

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchtext
import torch_xla
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from torch.utils.data import Dataset, DataLoader, random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch_xla.core.xla_model as xm
from torchtext.datasets import AG_NEWS

print(torch.__version__)
print(torchtext.__version__)
print(torch_xla.__version__)

2.5.0.dev20240629+cpu
0.17.0.dev20240629+cpu
2.4.0+git3bdca4d


/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
# !pip install torchdata
data_dir = './data' # TODO
dataset, test_data = AG_NEWS(root=data_dir)

/content/torchdata/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [ ]:
next(iter(dataset))
next(iter(test_data))

(3,
 "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")

In [ ]:
tokenizer = get_tokenizer('basic_english')
tokenizer('I love BITAmin!')

['i', 'love', 'bitamin', '!']

In [ ]:
def yield_tokens(data):
  for _, text in data:
    yield tokenizer(text)

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:365: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn(


In [ ]:
def text_preprocess(x):
  return vocab(tokenizer(x))

In [ ]:
def label_preprocess(x):
  return int(x) - 1

In [ ]:
dataset = torchtext.datasets.AG_NEWS(root=data_dir, split='test')

In [ ]:
TRAIN_AG_NEWS_PATH = 'data/datasets/AG_NEWS/train.csv' # TODO
TEST_AG_NEWS_PATH = 'data/datasets/AG_NEWS/test.csv'# TODO

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path: str ='./data/AG_NEWS/train.csv', train=True):
        tqdm_notebook.pandas(desc="PROGRESS>>")
        self.data = pd.read_csv(path, sep=',', header=None, names=['class','title','description'])
        self.train = train
        self.path = path

        data = self.data['title'] + ' ' + self.data['description']
        self.X = list()
        for line in data:
          self.X.append(line)
        self.y = self.data['class']

        self.classes = ['World', 'Sports', 'Business', 'Sci/Tech']

    def __len__(self):
        len_dataset = None
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = None, None
        X = self.X[idx]
        if self.train is True:
            y = self.y[idx]
        return y, X

    def split_dataset(self, val_ratio = 0.2):
        data_size = len(self)
        val_set_size = int(data_size * val_ratio)
        train_set_size = data_size - val_set_size

        train_set, val_set = random_split(self, [train_set_size, val_set_size])

        return train_set, val_set

In [ ]:
dataset = CustomDataset(TRAIN_AG_NEWS_PATH, train=True)
train_dataset, val_dataset = dataset.split_dataset(0.2)

In [ ]:
next(iter(train_dataset))

(2,
 'Myskina beats Dementieva to win in Moscow In a rematch of All-Russian French Open championship match, Anastasia Myskina trounced Elena Dementieva 7-5 and 6-0 to win the Kremlin Cup final to retain her Moscow title on Sunday.')

In [ ]:
next(iter(val_dataset))

(2,
 'Henman fires amid models furore Tim Henman, in common with Michael Owen, has found the target here at last. The British No 1, beaten in his opening matches on two previous visits to the Madrid Masters, defeated Albert Costa, of Spain, yesterday, 6-4, 6-2.')

In [ ]:
len(train_dataset), len(val_dataset)

(96000, 24000)

In [ ]:
text_preprocess(train_dataset[0][1])

[6016,
 1808,
 5798,
 4,
 109,
 7,
 930,
 7,
 5,
 7585,
 6,
 11100,
 432,
 149,
 417,
 620,
 3,
 8250,
 6016,
 14217,
 5386,
 5798,
 3878,
 8,
 4612,
 4,
 109,
 2,
 3590,
 174,
 216,
 4,
 4174,
 256,
 930,
 391,
 10,
 91,
 1]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
print(xm.xrt_world_size())
device

1


device(type='xla', index=0)

In [ ]:
def collate_batch(batch):
    labels, texts, offsets = [], [], [0]
    for (label, text) in batch:
        labels.append(label_preprocess(label))
        processed_text = torch.tensor(text_preprocess(text), dtype=torch.int64)
        texts.append(processed_text)
        offsets.append(processed_text.size(0))

    labels = torch.tensor(labels, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    texts = torch.cat(texts)
    return labels.to(device), texts.to(device), offsets.to(device)

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.RandomSampler(val_dataset)

In [ ]:
batch_size = 128

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_classes = num_classes

        self.embedding = nn.EmbeddingBag(self.vocab_size, self.embed_dim, sparse=False)

        self.rnn = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_dim, self.num_classes)

    def forward(self, text, offsets):

        embedded = self.embedding(text, offsets).view(batch_size, -1, self.embed_dim)

        hidden = torch.zeros(self.num_layers, embedded.size(0), self.hidden_dim).to(device)

        rnn_out, hidden= self.rnn(embedded, hidden)
        out = self.linear(rnn_out[:, -1:]).view([-1,self.num_classes])
        return out

In [ ]:
vocab_size = len(vocab)
embed_dim = 64
hidden_dim = 32
num_layers = 1
num_classes = 4
learning_rate = 0.5
epochs = 5
criterion = torch.nn.CrossEntropyLoss()
log_interval = 100

model = TextClassifier(vocab_size, embed_dim, hidden_dim, num_layers, num_classes).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 2, gamma=0.5)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch, sampler=train_sampler ,drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch, sampler=test_sampler ,drop_last=True)

In [ ]:
def train(dataloader, epoch):
    model.train()
    train_acc = 0
    train_count = 0
    for idx, (labels, texts, offsets) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        labels, texts, offsets = labels.to(device), texts.to(device), offsets.to(device)

        outs = model(texts, offsets)
        predicts = torch.argmax(outs, dim=-1)

        loss = criterion(outs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 5.0) # TODO
        #optimizer.step()
        xm.optimizer_step(optimizer, barrier=True) # If you use TPU

        train_acc += (predicts == labels).sum().item()
        train_count += labels.size(0)

        if idx % log_interval == 0 and idx > 0:
            print('| epoch {:3d} | {:5d}/{:5d} batches | loss {:8.3f} | accuracy {:8.3f}'.format(epoch, idx, len(dataloader), loss, train_acc / train_count))

    scheduler.step()

def evaluate(dataloader):
    model.eval()
    val_acc = 0
    val_count = 0
    val_acc_items = []
    with torch.no_grad():
        for idx, (labels, texts, offsets) in enumerate(dataloader):
            labels, texts, offsets = labels.to(device), texts.to(device), offsets.to(device)
            outs = model(texts, offsets)
            predicts = torch.argmax(outs, dim=-1)
            acc_item = (labels == predicts).sum().item()
            val_acc_items.append(acc_item)
            val_count += labels.size(0)
            val_acc = np.sum(val_acc_items) / val_count
    return val_acc

In [ ]:
total_acc = 0
for epoch in range(1, epochs + 1):
    train(train_dataloader, epoch)
    acc_val = evaluate(val_dataloader)

    if total_acc < acc_val:
        total_acc = acc_val

    print('-' * 60)
    print('| end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc))
    print('-' * 60)

103it [00:09, 19.19it/s]

| epoch   1 |   100/  750 batches | loss    1.341 | accuracy    0.338


204it [00:15, 17.13it/s]

| epoch   1 |   200/  750 batches | loss    1.220 | accuracy    0.393


304it [00:22, 17.94it/s]

...

| epoch   5 |   600/  750 batches | loss    0.733 | accuracy    0.705


704it [00:39, 18.24it/s]

| epoch   5 |   700/  750 batches | loss    0.745 | accuracy    0.706


750it [00:41, 17.88it/s]


------------------------------------------------------------
| end of epoch   5 | valid accuracy    0.709 
------------------------------------------------------------


In [ ]:
test_dataset = CustomDataset(TEST_AG_NEWS_PATH, train=False)
test_dataloader =  DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch,drop_last=True)

In [ ]:
acc_val = evaluate(test_dataloader)
print('-' * 59)
print('test accuracy {:8.3f} '.format(acc_val))
print('-' * 59)

-----------------------------------------------------------
test accuracy    0.709 
-----------------------------------------------------------
